In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, StackingRegressor
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_percentage_error

In [72]:
dataset = pd.read_csv("/Users/alert/Downloads/C-PEPTIDE/diabetes_data 2.csv")
dataset_label = dataset["Insulin_Levels"]
dataset.drop(columns=["Insulin_Levels","Unnamed: 0"], inplace=True)
dataset_new = pd.get_dummies(dataset, columns=['Family_History_of_Diabetes', 'Gestational_Diabetes',
'Physical_Activity', 'Smoking', 'Alcohol_Consumption',
       'Obesity', 'Diet', 'Sleep_Apnea', 'Diabetes_Status', 'Hypertension', 'PCOS'])
dataset_new

,Fasting_Blood_Glucose,Postprandial_Blood_Glucose,HbA1c,Random_Blood_Glucose,BMI,Waist_Circumference,Triglyceride_Levels,Blood_Pressure_Systolic,Blood_Pressure_Diastolic,LDL_Cholesterol,...,Diet_No,Diet_Yes,Sleep_Apnea_No,Sleep_Apnea_Yes,Diabetes_Status_Negative,Diabetes_Status_Positive,Hypertension_No,Hypertension_Yes,PCOS_No,PCOS_Yes
0,118.690215,111.472598,5.570234,224.721689,29.299897,37.872710,212.064239,93.491951,100.418755,188.765141,...,True,False,True,False,False,True,False,True,True,False
1,193.592860,172.123161,5.481873,253.236721,35.135808,39.468713,93.096591,106.809528,98.109810,138.296863,...,False,True,True,False,True,False,True,False,True,False
2,165.159212,228.400792,9.437527,127.607617,34.004023,47.090948,268.098641,164.812122,56.702794,164.238335,...,True,False,True,False,False,True,True,False,True,False
3,147.825603,204.478231,5.497277,213.721043,18.847498,36.800088,203.279060,159.009152,114.580068,126.686279,...,True,False,False,True,False,True,False,True,False,True
4,90.282423,217.115395,5.631698,201.501576,18.731237,47.392994,89.300971,121.557842,89.793054,151.904495,...,False,True,False,True,False,True,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,81.731503,132.332546,5.712814,142.634373,31.867784,33.039740,127.180164,102.211989,82.196275,91.615905,...,True,False,True,False,True,False,True,False,False,True
2996,101.889512,122.206950,5.423619,178.741817,31.798141,34.648423,139.123233,118.578609,74.112063,82.140444,...,True,False,False,True,True,False,False,True,True,False
2997,93.294244,130.571657,5.311118,119.174624,24.852150,39.421332,112.996152,128.685327,71.592304,95.511762,...,True,False,False,True,True,False,True,False,False,True
2998,100.005629,124.054126,5.145150,158.280116,26.359585,28.989608,122.985922,104.613596,74.805198,81.549216,...,True,False,False,True,True,False,False,True,True,False


In [167]:
X = dataset_new
y = dataset_label

scaler = StandardScaler()
X_another = scaler.fit_transform(X)
print(X_another)

model = Lasso(alpha=0.01)
model.fit(X_another, y)
selected_features = X.columns[model.coef_ != 0]
X_new = X[selected_features]


X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)


base_models = [
    ("rdf", RandomForestRegressor(n_estimators=100, max_features="sqrt", min_samples_leaf=2, max_depth=5, random_state=42)),
    ("kn", KNeighborsRegressor(n_neighbors=5, weights="distance")),
    ("xgb", XGBRegressor(n_estimators=300, learning_rate=0.1, subsample=1.0, max_depth=5, random_state=42))
]


meta_model = LinearRegression()

stacked_model = StackingRegressor(estimators=base_models,
                                   final_estimator=meta_model, passthrough=True)

xgbmodel = Pipeline(steps=[
    ("model", stacked_model)
])

xgbmodel.fit(X_train, y_train)
prediction = xgbmodel.predict(X_test)

print(root_mean_squared_error(y_test, prediction))
print(mean_absolute_percentage_error(y_test, prediction))


target = y_test  # replace 'target' with your column name

min_val = target.min()
max_val = target.max()
range_val = max_val - min_val

print("Minimum:", min_val)
print("Maximum:", max_val)
print("Range:", range_val)

# Optional: calculate RMSE as percentage of range
rmse = root_mean_squared_error(y_test, prediction)
rmse_percent = (rmse / range_val) * 100
print(f"RMSE is {rmse_percent:.2f}% of the target range")


[[-0.24761415 -1.31884062 -0.83605613 ...  0.9966722   0.96914252
  -0.96914252]
 [ 1.76995418 -0.23469595 -0.8946768  ... -1.00333891  0.96914252
  -0.96914252]
 [ 1.00406898  0.77128141  1.72958432 ... -1.00333891  0.96914252
  -0.96914252]
 ...
 [-0.93167689 -0.97743995 -1.00795917 ... -1.00333891 -1.03183998
   1.03183998]
 [-0.75089986 -1.09394252 -1.11806555 ...  0.9966722   0.96914252
  -0.96914252]
 [-0.9729521  -0.96039279 -0.86554188 ... -1.00333891 -1.03183998
   1.03183998]]
10.344487335764546
0.6459130268562637
Minimum: 2.0
Maximum: 49.81504788480024
Range: 47.81504788480024
RMSE is 21.63% of the target range


In [ ]:
10.308056702074976
0.6526884534074991
Minimum: 2.0
Maximum: 49.81504788480024
Range: 47.81504788480024
RMSE is 21.56% of the target range